Test noun phrase extraction in spacy.

In [1]:
import spacy
import sys
sys.path.append('./code/pipeline')
from ling_preprocessing import *
nlp = spacy.load("/usr/local/lib/python3.6/dist-packages/en_core_web_sm/en_core_web_sm-2.0.0")

In [2]:
doc = "The first sentence is a pure test. The second is a grey mouse. Water is interesting."
doc2 = "The first sentence is a pure test . The second is a grey mouse .".split(' ')
print([(doc2[i], i) for i in range(len(doc2))])

[('The', 0), ('first', 1), ('sentence', 2), ('is', 3), ('a', 4), ('pure', 5), ('test', 6), ('.', 7), ('The', 8), ('second', 9), ('is', 10), ('a', 11), ('grey', 12), ('mouse', 13), ('.', 14)]


In [3]:
nlp_doc = nlp(doc)

In [4]:
for np in nlp_doc.noun_chunks:
    head_idx = np.root.i-np.start
    print(np, np.start, np.root.i, ':', head_idx, np[head_idx])

The first sentence 0 2 : 2 sentence
a pure test 4 6 : 2 test
The second 8 9 : 1 second
a grey mouse 11 13 : 2 mouse
Water 15 15 : 0 Water


Wihtout sentence segregation:

In [5]:
nps = []
doc_tokens = [t.text for t in nlp_doc]
print(doc_tokens)
for np in nlp_doc.noun_chunks:
    np = nlp_doc[np.start: np.end]
    np_concat = '_'.join(np.text.split(' '))
    nps.append((np_concat, np.start, np.end))
print(nps)
for n in nps[::-1]:
    print(n[0])
    print(doc_tokens[n[1]:n[2]])
    doc_tokens[n[1]:n[2]] = [n[0]]
print(doc_tokens)

['The', 'first', 'sentence', 'is', 'a', 'pure', 'test', '.', 'The', 'second', 'is', 'a', 'grey', 'mouse', '.', 'Water', 'is', 'interesting', '.']
[('The_first_sentence', 0, 3), ('a_pure_test', 4, 7), ('The_second', 8, 10), ('a_grey_mouse', 11, 14), ('Water', 15, 16)]
Water
['Water']
a_grey_mouse
['a', 'grey', 'mouse']
The_second
['The', 'second']
a_pure_test
['a', 'pure', 'test']
The_first_sentence
['The', 'first', 'sentence']
['The_first_sentence', 'is', 'a_pure_test', '.', 'The_second', 'is', 'a_grey_mouse', '.', 'Water', 'is', 'interesting', '.']


With sentence segretation:

In [6]:
import re
# doc_sents_tokens = [token.text for sent in nlp_doc.sents for token in sent]
doc_sents = []
for sent in list(nlp_doc.sents)[::-1]:
    s = []
    for word in sent:
        s.append(word.text)
    
    nps = []
    for np in list(sent.noun_chunks)[::-1]:
        np_slice = sent[np.start-sent.start: np.end-sent.start]
        # print(np_slice, np_slice[0], type(np_slice), type(np_slice[0]))
        add = 0
        if np_slice[0].text in ['a', 'A', 'the', 'The']:
            add += 1
            np_slice = np_slice[1:]
        np_concat = '_'.join(np_slice.text.split(' '))
        # print(repr(np), np.start, np.end, sent.start, np.start-sent.start, np.end-sent.start)
        s[np.start-sent.start+add: np.end-sent.start] = [np_concat]
        # print(s)
        nps.append((np_concat, np.start, np.end))
    print(nps)
    print(30*'-')
    
    doc_sents.append(s)
    
print(doc_sents[::-1])

[('Water', 15, 16)]
------------------------------
[('grey_mouse', 11, 14), ('second', 8, 10)]
------------------------------
[('pure_test', 4, 7), ('first_sentence', 0, 3)]
------------------------------
[['The', 'first_sentence', 'is', 'a', 'pure_test', '.'], ['The', 'second', 'is', 'a', 'grey_mouse', '.'], ['Water', 'is', 'interesting', '.']]


In [7]:
pp = DBLPLingPreprocessor('.', '.', "/usr/local/lib/python3.6/dist-packages/en_core_web_sm/en_core_web_sm-2.0.0")
pp._process_doc(doc, True)
print(pp._pp_corpus)

[[[('The', 'DT', 'the', False), ('first_sentence', '1np', 'first_sentence', False), ('is', 'VBZ', 'be', True), ('a', 'DT', 'a', True), ('pure_test', '1np', 'pure_test', False), ('.', '.', '.', False)], [('The', 'DT', 'the', False), ('second', '0np', 'second', False), ('is', 'VBZ', 'be', True), ('a', 'DT', 'a', True), ('grey_mouse', '1np', 'grey_mouse', False), ('.', '.', '.', False)], [('Water', '0np', 'water', False), ('is', 'VBZ', 'be', True), ('interesting', 'JJ', 'interesting', False), ('.', '.', '.', False)]]]
